In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
from matplotlib.colors import ListedColormap
from skimage import exposure
from skimage.io import imread, imsave

%matplotlib inline
import cupy as cp
from dexp.processing.restoration import dehazing
from dexp.utils.backends import Backend, CupyBackend, NumpyBackend, best_backend
from tqdm import tqdm

cp.cuda.set_allocator(cp.cuda.MemoryPool(cp.cuda.malloc_managed).malloc)
import os

In [3]:
def create_MIP(im_proj):
    im_proj = skimage.exposure.rescale_intensity(im_proj, out_range=(0, 1))
    vmin_green, vmax_green = np.percentile(im_proj[0, :, :], q=(0.5, 99.5))
    clipped_green = exposure.rescale_intensity(
        im_proj[0, :, :], in_range=(vmin_green, vmax_green), out_range=np.float32
    )

    vmin_magenta, vmax_magenta = np.percentile(im_proj[1, :, :], q=(0.5, 99.5))
    clipped_magenta = exposure.rescale_intensity(
        im_proj[1, :, :], in_range=(vmin_magenta, vmax_magenta), out_range=np.float32
    )

    channel1 = green_matplotlib_map(clipped_green)
    channel2 = magenta_matplotlib_map(clipped_magenta)
    assembled = np.stack((channel1, channel2), axis=3)
    newim = np.max(assembled, axis=3)
    return newim

In [ ]:
t = 455
output = "/cluster/work/treutlein/DATA/imaging/viventis/His-GFP_memRFP_WTC_HB4_D4_Eb22-03-21/Position_1_Settings_1/denoised/"
stack_mcherry = imread(output + "mCherry/t" + f"{t:04}" + "_mCherry.tif")
stack_gfp = imread(output + "GFP/t" + f"{t:04}" + "_GFP.tif")
stack_mcherry = dehaze_downscale(
    cp.asarray(stack_mcherry.astype(np.float32)), size=50, minimal_zero_level=0
).get()
stack_gfp = dehaze_downscale(
    cp.asarray(stack_gfp.astype(np.float32)), size=50, minimal_zero_level=0
).get()

image = np.stack([stack_gfp, stack_mcherry], axis=0)
print(np.max(image))

ims = []
colors = ["Red", "Magenta"]
green_map = [[0, i / 255, 0] for i in range(256)]
green_matplotlib_map = ListedColormap(green_map, "Green")
magenta_map = [[i / 255, 0, i / 255] for i in range(256)]
magenta_matplotlib_map = ListedColormap(magenta_map, "Magenta")


im_proj = np.max(image, axis=1)
im_proj = skimage.exposure.rescale_intensity(im_proj, out_range=(0, 1))
vmin_green, vmax_green = np.percentile(im_proj[0, :, :], q=(0.5, 99.5))
clipped_green = exposure.rescale_intensity(
    im_proj[0, :, :], in_range=(vmin_green, vmax_green), out_range=np.float32
)

vmin_magenta, vmax_magenta = np.percentile(im_proj[1, :, :], q=(0.5, 99.5))
clipped_magenta = exposure.rescale_intensity(
    im_proj[1, :, :], in_range=(vmin_magenta, vmax_magenta), out_range=np.float32
)
channel1 = green_matplotlib_map(clipped_green)
channel2 = magenta_matplotlib_map(clipped_magenta)

assembled = np.stack((channel1, channel2), axis=3)
newim = np.max(assembled, axis=3)

fig, axes = plt.subplots(1, figsize=(20, 20))
axes.imshow(newim)
plt.show()

In [7]:
images_folder = "/cluster/project/treutlein/DATA/imaging/viventis/His-GFP_memRFP_WTC_HB4_D4_Eb22-03-21_processed/Position_1_Settings_1_Processed/dehazed/"
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

In [11]:
channels = ["GFP", "mCherry"]
output = "/cluster/work/treutlein/DATA/imaging/viventis/His-GFP_memRFP_WTC_HB4_D4_Eb22-03-21/Position_1_Settings_1/denoised/"

In [ ]:
for t in tqdm(range(1, 699)):
    for channel in tqdm(channels):
        stack = imread(output + f"{channel}/t{t:04}_{channel}.tif").copy()
        with CupyBackend():
            xp = Backend.get_xp_module()
            print(xp)
            stack_dehazed = dehazing.dehaze(
                cp.asarray(stack).astype(np.float32),
                size=60,
                minimal_zero_level=0,
                correct_max_level=False,
            )
            stack_dehazed = Backend.to_numpy(stack_dehazed)
        output_path = f"{images_folder}{channel}/"
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        imsave(
            output_path + f"t{t:04}_{channel}.tif",
            stack_dehazed.clip(0.0, 2**16 - 1).astype(np.uint16),
            plugin="tifffile",
            check_contrast=False,
            compress=6,
            bigtiff=True,
        )